In [1]:
import pandas as pd
# import stanza
from text_mining.text_processing.text2bow import Text2BowTransformerPhrases
from text_mining.text_processing.tokenizer import Tokenizer

In [2]:
directory = '~/PycharmProjects/tfm_hugopobil'
df = pd.read_csv(f'{directory}/data/tweets_nlp_modelling.csv')
btc_usd_grouped = pd.read_csv(f'{directory}/data/btc_usd_grouped.csv')
df = df.set_index('date_clean')

In [3]:
# stanza.download('en')

In [ ]:
tokenizer = Tokenizer(tokenizer_model='stanza', language='en')
text2bow = Text2BowTransformerPhrases(tokenizer=tokenizer, form_phrases=False)

results = []
count = 0

for ind, row in df.iterrows():
    results.append(text2bow.fit_transform(str(row.cleaned_tweets)))
    count +=1
    if count % 100 == 0:
        print(count)

100
200
300
400
500
600
700
800
900
1000
1100
1200
1300
1400
1500
1600
1700
1800
1900
2000
2100
2200
2300
2400
2500
2600
2700
2800
2900
3000
3100
3200
3300
3400
3500
3600
3700
3800
3900
4000
4100
4200
4300
4400
4500
4600
4700


In [ ]:
df['bow'] = results
print(df.shape)
df.head()

In [ ]:
df.to_csv(f'{directory}/data/tweets_nlp_bow.csv')